In [1]:
from torch.utils.data import Dataset
import pydicom
import numpy as np
import pandas as pd

import pickle
import os
import time

In [2]:
target = 'Adjuvant Chemotherapy'

Load image data using PyTorch Dataset

In [3]:
class dcm_npy_loader(Dataset):

    def __init__(self, img_dir, load, shape=(512, 512)):
        self.img_dir = img_dir
        self.load = load
        self.shape = shape

        if not self.load:
            load_paths = list()
            for (dirpath, dirnames, filenames) in os.walk(img_dir):
                load_paths += [os.path.join(dirpath, file) for file in filenames]

            ids = np.array([], dtype=np.int8)
            img_list = []
            num_exceptions = 0
            for path in load_paths:
                try:
                    file = pydicom.dcmread(path)
                    if file.pixel_array.shape == self.shape:
                        id = file.PatientID
                        for c in id:
                            if not c.isdigit():
                                id = id.replace(c, '')

                        ids = np.append(ids, id)

                        subject_dict = {
                            'one image': torchio.ScalarImage(path),
                            'id':id,
                            'SliceLocation':file.SliceLocation
                        }
                        subject = torchio.Subject(subject_dict)

                        img_list.append(subject)
                except:
                    if num_exceptions < 5:
                        print("Image " + path + " could not be loaded")
                    elif num_exceptions == 5:
                        print("More than 5 exceptions occured")
                    
                    num_exceptions = num_exceptions + 1

            img_list.sort(key=lambda s: int(s['id']))

            ids = set([int(s['id']) for s in img_list])

            all_img3d = []
            all_ids = []
            all_sliceLocs = []
            for id in ids:
                prev_time_id = time.time()

                slices = []
                i = 0
                for slice in img_list:

                    p_id = int(slice['id'])

                    if int(p_id) == int(id):
                            
                        slices.append(slice)

                        # remove slices from dataset that have already been appended
                        del img_list[i]

                    i = i + 1

                id_slices = torchio.SubjectsDataset(slices)

                # ensure slices are in the correct order
                id_slices = sorted(id_slices, key=lambda s: s.SliceLocation)

                # create 3D array
                img_shape = list(id_slices[0]['one image']['data'].shape)
                img_shape.append(len(id_slices))
                img3d = np.empty(img_shape, dtype=np.int8)

                p_id = id_slices[0]['id']
                # get only numbers from patient id
                p_id = [int(s) for s in str(p_id) if s.isdigit()]
                p_id = int(''.join([str(i) for i in p_id]))

                slice_locs = []

                # fill 3D array with the images from the files
                for i, s in enumerate(id_slices):
                    img2d = s['one image']['data']

                    slice_locs.append(s['SliceLocation'])
                    if list(img2d.shape) == img_shape[:4]:
                        img3d[:, :, :, :, i] = img2d

                all_ids.append(p_id)
                img3d = np.squeeze(img3d)
                all_img3d.append(img3d)
                all_sliceLocs.append(slice_locs)

                aft_time_id = time.time()
                id_load_time = aft_time_id - prev_time_id

                print("id: " + str(id) + " has completed " + "in " + str(round(id_load_time,0)) + " seconds!")

            self.data = [all_img3d, all_ids, all_sliceLocs]

            with open('C:\\Users\\trist\\cs_projects\\Cancer_Project\\Cancer_ML\\data\\Duke-Breast-Cancer-MRI\\data', 'wb') as fp:
                pickle.dump(self.data, fp)
        
        else:
            with open('C:\\Users\\trist\\cs_projects\\Cancer_Project\\Cancer_ML\\data\\Duke-Breast-Cancer-MRI\\data', 'rb') as fp:
                self.data = pickle.load(fp)

    def __len__(self):
        return len(self.data[0])

    def __getitem__(self, idx):
        sample = [self.data[0][idx], self.data[1][idx], self.data[2][idx]]
        return sample

In [4]:
def import_numpy_2d(path, clinical_ids):
    ds = dcm_npy_loader(path, load=True)

    patients = []
    ids = []
    sliceLocs = []
    for sample in ds:
        patients.append(sample[0])
        ids.append(sample[1])
        sliceLocs.append(sample[2])

    i = 0
    for p in patients:
        p_id = ids[i]
        if not p_id in clinical_ids:
            del patients[i]

        i = i + 1

    slice_locations_min = []
    slice_locations_max = []
    for sl in sliceLocs:
        mi = np.amin(sl)
        ma = np.amax(sl)
        slice_locations_min.append(mi)
        slice_locations_max.append(ma)

    # number of intervals to collect for each patient
    interval_num = 100

    interval_nums = []
    all_intervals = []
    for ma, mi in zip(slice_locations_max, slice_locations_min):
        subinterval_length = int(round((ma-mi)/interval_num, 0))
        interval_nums.append(interval_num)

        interval_num = min(interval_nums)

        # set min slice loc as initial interval marker
        intervals = []
        interval_marker = mi
        for i in range(interval_num):
            intervals.append(interval_marker)
            interval_marker = interval_marker+subinterval_length*i

        all_intervals.append(intervals)

    # array to store all intervals
    all_interval_imgs = np.empty(shape=(len(ids), len(intervals), 512, 512), dtype=np.float16)

    # reshape all patients into (num_slices, res1, res2)
    new_patients = []
    for patient in patients:
        patient = np.reshape(patient, (patient.shape[-1], patient.shape[-2], patient.shape[-3]))
        new_patients.append(patient)

    patients = new_patients

    m = 0
    for p_id in ids:
        for i in range(len(intervals)):
            if i < (len(intervals)-1):
                low = intervals[i]
                high = intervals[i+1]

                interval_imgs = np.empty(shape=(len(intervals), 512, 512), dtype=np.float16)

                j = 0
                for p in patients:
                    id = ids[j]

                    k = 0
                    if int(id) == int(p_id):

                        for s in p:
                            slice_location=sliceLocs[j][k]

                            if slice_location < high and slice_location > low:
                                interval_imgs[i] = s

                                # break after adding one image from the interval
                                break

                            k = k + 1
                    
                    j = j + 1

        m = m + 1

    return all_interval_imgs, ids

In [5]:
df = pd.read_csv('..\Clinical and Other Features.csv')

In [6]:
clinical_ids = df[list(df.columns)[0]]

In [7]:
df = df.set_index(str(list(df.columns)[0]))

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer

def tokenize_dataset(pd_dataset):

    # replace missing spots with a string then after dataset is encoded, replace with mean of column
    df = pd_dataset.fillna('empty')

    if df.shape[0] >= df.shape[1]:
        long_axis = df.shape[0]
        short_axis = df.shape[1]
    else:
        long_axis = df.shape[1]
        short_axis = df.shape[0]

    word_list = []
    for i in range(long_axis):
        for n in range(short_axis):
            
            if long_axis == df.shape[0]:
                data = df.iloc[i, n]
            else:
                data = df.iloc[n, i]

            if str(type(data)) == "<class 'str'>":

                # list of chars to be removed from data
                char_blocked = [' ', '.', '/', '-', '_', '>', '+', ',', ')', '(', '*',
                                '=', '?', ':', '[', ']', '#', '!', '\n', '\\', '}',
                                '{', ';', '%', '"']
                
                for char in char_blocked:
                    if char in data:
                        data = data.replace(char, '')

                data = data.lower()

                if long_axis == df.shape[0]:
                    df.iloc[i, n] = data
                else:
                    df.iloc[n, i] = data

                word_list.append(data)

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(word_list)
    code_dict = tokenizer.word_index

    for i in range(long_axis):
        for n in range(short_axis):
            if long_axis == df.shape[0]:
                data = df.iloc[i, n]
            else: 
                data = df.iloc[n, i]

            if str(type(data)) == "<class 'str'>":
                
                data = int(code_dict[data])

            if long_axis == df.shape[0]: 
                df.iloc[i, n] = data
            else: 
                df.iloc[n, i] = data

    # replace spots previously denoted as 'empty' with mean of column
    for column in list(df.columns):
        col = df[column].copy()

        empty_indices = []

        i = 0
        for val in col:
            if val == code_dict['empty']:
                empty_indices.append(i)

            i = i + 1

        # get series labels at the empty indices for .drop function
        col_labels = list(col.index)

        empty_labels = []
        for index in empty_indices:
            empty_labels.append(col_labels[index])

        col_without_empty = col.drop(empty_labels)

        col_mean = col_without_empty.mean()

        i = 0
        for val in col:
            if val == code_dict['empty']:
                col[i] = col_mean

            i = i + 1
        
        df[column] = col

    # convert all cols to numeric vals
    df = df.astype('int64')

    return df

In [9]:
df = tokenize_dataset(df)

In [10]:
img_array, image_ids = import_numpy_2d(None, list(clinical_ids))

In [11]:
from sklearn.model_selection import train_test_split

train_ids, test_ids = train_test_split(image_ids, test_size=0.2, random_state=84)
test_ids, val_ids = train_test_split(test_ids, test_size=0.5, random_state=84)

In [12]:
filtered_df = df.loc[image_ids]

In [13]:
y = filtered_df[target]

In [14]:
filtered_df = filtered_df.drop(target, axis=1)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(img_array, y, test_size=0.2, random_state=84)

# split test data into validation and test
X_test, X_val = train_test_split(X_test, test_size=0.5, random_state=84)
y_test, y_val = train_test_split(y_test, test_size=0.5, random_state=84)

normalize data

In [16]:
import tensorflow as tf
def four_dim_normalization(img_array):
    i = 0
    for p in img_array:
        s = tf.image.per_image_standardization(p)
        img_array[i] = s
        i = i + 1
        
    return img_array

In [17]:
X_train = four_dim_normalization(X_train)
X_test = four_dim_normalization(X_test)
X_val = four_dim_normalization(X_val)

In [18]:
X_train = tf.image.random_crop(X_train, (X_train.shape[0], X_train.shape[1], 256, 256))
X_test = tf.image.random_crop(X_test, (X_test.shape[0], X_test.shape[1], 256, 256))
X_val = tf.image.random_crop(X_val, (X_val.shape[0], X_val.shape[1], 256, 256))

In [19]:
import torch

def grey_to_rgb(img_array):
    width = img_array.shape[-3]
    height = img_array.shape[-2]
    new_img_array = torch.empty((img_array.shape[0], img_array.shape[1], width, height, 3), dtype=torch.uint8)
    i = 0
    for p in img_array:
        new_p = torch.empty((p.shape[0], width, height, 3), dtype=torch.float16)
        j = 0
        for s in p:
            s = torch.squeeze(s)
            out = torch.empty((width, height, 3), dtype=torch.float16)
            out[:, :, 0] = s
            out[:, :, 1] = s
            out[:, :, 2] = s

            new_p[j] = out

            j = j + 1

        new_img_array[i] = new_p

        i = i + 1
    
    return new_img_array

In [20]:
X_train = torch.from_numpy(np.expand_dims(X_train, -1))
X_test = torch.from_numpy(np.expand_dims(X_test, -1))
X_val = torch.from_numpy(np.expand_dims(X_val, -1))

C:\Users\trist\AppData\Local\Temp\ipykernel_23172\3800326807.py:1: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  X_train = torch.from_numpy(np.expand_dims(X_train, -1))


Define model

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
import torch.nn as nn

class torch_cnn(nn.Module):
    def __init__(self, num_classes):
        super(torch_cnn, self).__init__()
        self.num_classes = num_classes
        self.to(device)
        self.res = None
        self.fc1 = nn.Linear(400, 1)

    def forward(self, x):
        x = self.res(x)
        x = self.fc1(x)

        return x

    def train_func(self, config, data):
        id_X_train = data[0]
        id_y_train = data[1]
        self.res = data[2]
        X_train = ray.get(id_X_train)
        y_train = ray.get(id_y_train)
        epochs = config['epochs']
        batch_size = config['batch_size']
        lr = config['lr']
        criterion = torch.nn.CrossEntropyLoss()
        for epoch in range(int(epochs)):
            running_loss = 0.0
            for i in range((X_train.shape[0]-1)//batch_size + 1):
                start_i = i*batch_size
                end_i = start_i+batch_size

                xb = X_train[start_i:end_i]
                yb = y_train[start_i:end_i]

                xb = xb.to(device)
                yb = yb.to(device)

                xb = grey_to_rgb(xb)
                xb = xb/255

                xb = torch.reshape(xb, (xb.shape[0], xb.shape[-1], xb.shape[1], xb.shape[2], xb.shape[3]))
                pred = self(xb)

                optimizer = torch.optim.Adam(self.parameters(), lr=lr)

                loss = criterion(pred, yb)
        
                loss.backward()
                optimizer.step()

                # zero the parameter gradients
                optimizer.zero_grad()

                # print stats
                running_loss += loss.item()
                pred = pred.detach()
                self.loss = running_loss
                pred = np.argmax(pred, axis=1)
                self.accuracy = accuracy_score(yb, pred)
                self.f1_score = f1_m(yb, pred)
                self.recall = recall_m(yb, pred)
                self.balanced_acc = balanced_accuracy_score(yb, pred)
                if i % 2000 == 1999: # print every 2000 mini-batches
                    print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}', 'Accuracy: %.4f' %self.accuracy, 'F1: %.4f' %self.f1_score, 'Recall: %.4f' %self.recall, 'Balanced Accuracy: %.4f' %self.balanced_acc)
                tune.report(loss=running_loss, accuracy=self.accuracy)
                running_loss = 0.0

        print("Finished Training")

In [23]:
import torchvision.models as models
import ray
from ray import tune
from ray.tune.schedulers.async_hyperband import ASHAScheduler

class cnn:

    def __init__(self, load_model=True):
        self.load_model=load_model
        self.res = models.video.r3d_18(pretrained=False)

    def main(self, X_train, y_train, num_samples=10, max_num_epochs=10, gpus_per_trial=2):
        X_train = X_train.type(torch.int8)
        
        y_train = torch.from_numpy(np.array(y_train))

        # get number of classes in y
        y = []
        for i in y_train:
            i = int(i)
            y.append(i)

        y = list(set(y))
        num_classes = len(y)

        # make classes start from 0
        class_dict = dict()
        for i in range(num_classes):
            class_dict[y[i]] = i

        i = 0
        for val in y_train:
            new_val = class_dict[int(val)]
            y_train[i] = new_val
            i = i + 1

        if len(y_train.shape) > 1:
            self.multi_target = True
        else:
            self.multi_target = False

        id_X_train = ray.put(X_train)
        id_y_train = ray.put(y_train)

        self.model = torch_cnn(num_classes)
        self.model.to(device)

        config = {
            'epochs':tune.choice([50, 100, 150]),
            'batch_size':tune.choice([8, 16, 20]),
            'lr':tune.loguniform(1e-4, 1e-1),
        }
        scheduler = ASHAScheduler(
            max_t=max_num_epochs,
            grace_period=1,
            reduction_factor=2
        )
        if torch.cuda.is_available():
            result = tune.run(
                tune.with_parameters(self.model.train_func, data=[id_X_train, id_y_train, self.res]),
                resources_per_trial={"cpu":4, "gpu":gpus_per_trial},
                config=config,
                metric="loss",
                mode="min",
                num_samples=num_samples,
                scheduler=scheduler
            )
        else:
            result = tune.run(
                tune.with_parameters(self.model.train_func, data=[id_X_train, id_y_train, self.res]),
                resources_per_trial={"cpu":4},
                config=config,
                metric="loss",
                mode="min",
                num_samples=num_samples,
                scheduler=scheduler
            )

        best_trial = result.get_best_trial("loss", "min", "last")
        print("Best trial config: {}".format(best_trial.config))
        print("Best trial final validation loss: {}".format(
            best_trial.last_result["loss"]))
        print("Best trial final validation accuracy: {}".format(
            best_trial.last_result['accuracy']))

        self.model.train_func(config=best_trial.config)

        return self.model

    def test_model(self, X_test, y_test):
        self.criterion = torch.nn.CrossEntropyLoss()
        X_test = torch.unsqueeze(X_test, -1)
        X_test = grey_to_rgb(X_test)/255
        # reshape to have 3 channels
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[-1], X_test.shape[1], X_test.shape[2], X_test.shape[3]))
        if type(X_test) == np.ndarray:
            X_test = torch.from_numpy(X_test).type(torch.float)
        y_test = torch.from_numpy(np.array(y_test))
        with torch.no_grad():
            y_pred = self.model(X_test)
            confusion_matrix(y_test, y_pred, save_name="image_only_c_mat_torch")
            test_loss = self.criterion(y_pred, y_test)
            accuracy = accuracy_score(y_test, y_pred)
            f1_score = f1_m(y_test, y_pred)
            recall = recall_m(y_test, y_pred)
            balanced_acc = balanced_accuracy_score(y_test, y_pred)

        return test_loss, accuracy, f1_score, recall, balanced_acc

    def get_model(self, X_train=None, y_train=None, X_val=None, y_val=None, epochs=10, batch_size=32):

        if self.load_model:
            self.model = torch_cnn()
            self.model.res = self.res
            self.model.load_state_dict(torch.load("torch_cnn_model.pth"), strict=False)
        else:
            self.model = self.main(X_train, y_train)

        return self.model

In [24]:
model = cnn(load_model=False)

In [ ]:
model.get_model(X_train, y_train, X_val, y_val)

2022-07-12 06:49:58,244	WARNING function_runner.py:598 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2022-07-12 06:49:58,474	INFO trial_runner.py:803 -- starting train_func_22de3_00000
2022-07-12 06:49:58,525	ERROR syncer.py:119 -- Log sync requires rsync to be installed.


Trial name,status,loc,batch_size,epochs,lr
train_func_22de3_00000,RUNNING,127.0.0.1:9272,16,100,0.00651355
train_func_22de3_00001,PENDING,,8,100,0.0323547
train_func_22de3_00002,PENDING,,8,150,0.000631945
train_func_22de3_00003,PENDING,,8,50,0.000481588
train_func_22de3_00004,PENDING,,20,100,0.00949023
train_func_22de3_00005,PENDING,,20,100,0.000351162
train_func_22de3_00006,PENDING,,8,50,0.00306286
train_func_22de3_00007,PENDING,,16,150,0.0175318
train_func_22de3_00008,PENDING,,20,150,0.0719229
train_func_22de3_00009,PENDING,,16,150,0.0102132


2022-07-12 06:50:01,303	INFO trial_runner.py:803 -- starting train_func_22de3_00001


Trial name,status,loc,batch_size,epochs,lr
train_func_22de3_00000,RUNNING,127.0.0.1:9272,16,100,0.00651355
train_func_22de3_00001,RUNNING,127.0.0.1:16604,8,100,0.0323547
train_func_22de3_00002,PENDING,,8,150,0.000631945
train_func_22de3_00003,PENDING,,8,50,0.000481588
train_func_22de3_00004,PENDING,,20,100,0.00949023
train_func_22de3_00005,PENDING,,20,100,0.000351162
train_func_22de3_00006,PENDING,,8,50,0.00306286
train_func_22de3_00007,PENDING,,16,150,0.0175318
train_func_22de3_00008,PENDING,,20,150,0.0719229
train_func_22de3_00009,PENDING,,16,150,0.0102132


Trial name,status,loc,batch_size,epochs,lr
train_func_22de3_00000,RUNNING,127.0.0.1:9272,16,100,0.00651355
train_func_22de3_00001,RUNNING,127.0.0.1:16604,8,100,0.0323547
train_func_22de3_00002,PENDING,,8,150,0.000631945
train_func_22de3_00003,PENDING,,8,50,0.000481588
train_func_22de3_00004,PENDING,,20,100,0.00949023
train_func_22de3_00005,PENDING,,20,100,0.000351162
train_func_22de3_00006,PENDING,,8,50,0.00306286
train_func_22de3_00007,PENDING,,16,150,0.0175318
train_func_22de3_00008,PENDING,,20,150,0.0719229
train_func_22de3_00009,PENDING,,16,150,0.0102132


...

Trial name,status,loc,batch_size,epochs,lr
train_func_22de3_00000,RUNNING,127.0.0.1:9272,16,100,0.00651355
train_func_22de3_00001,RUNNING,127.0.0.1:16604,8,100,0.0323547
train_func_22de3_00002,PENDING,,8,150,0.000631945
train_func_22de3_00003,PENDING,,8,50,0.000481588
train_func_22de3_00004,PENDING,,20,100,0.00949023
train_func_22de3_00005,PENDING,,20,100,0.000351162
train_func_22de3_00006,PENDING,,8,50,0.00306286
train_func_22de3_00007,PENDING,,16,150,0.0175318
train_func_22de3_00008,PENDING,,20,150,0.0719229
train_func_22de3_00009,PENDING,,16,150,0.0102132



(train_func pid=9272)     trainable(config, **fn_kwargs)
(train_func pid=9272)   File "C:\Users\trist\AppData\Local\Temp\ipykernel_23172\1618879806.py", line 43, in train_func
(train_func pid=9272)   File "C:\Users\trist\cs_projects\Cancer_Project\Cancer_ML\env\lib\site-packages\torch\nn\modules\module.py", line 1051, in _call_impl
(train_func pid=9272)     return forward_call(*input, **kwargs)
(train_func pid=9272)   File "C:\Users\trist\AppData\Local\Temp\ipykernel_23172\1618879806.py", line 12, in forward
(train_func pid=9272)   File "C:\Users\trist\cs_projects\Cancer_Project\Cancer_ML\env\lib\site-packages\torch\nn\modules\module.py", line 1051, in _call_impl
(train_func pid=9272)     return forward_call(*input, **kwargs)
(train_func pid=9272)   File "C:\Users\trist\cs_projects\Cancer_Project\Cancer_ML\env\lib\site-packages\torchvision\models\video\resnet.py", line 229, in forward
(train_func pid=9272)     x = self.layer1(x)
(train_func pid=9272)   File "C:\Users\trist\cs_projects

2022-07-12 06:50:43,065	ERROR trial_runner.py:876 -- Trial train_func_22de3_00000: Error processing event.
NoneType: None
2022-07-12 06:50:43,121	WARNING tune.py:650 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


Result for train_func_22de3_00000:
  date: 2022-07-12_06-50-01
  experiment_id: 5ba502e1fdfc4aaeb27a11e7b0fd0e9c
  hostname: Tristens-PC
  node_ip: 127.0.0.1
  pid: 9272
  timestamp: 1657630201
  trial_id: 22de3_00000
  


(pid=) 2022-07-12 06:50:43,332	INFO context.py:67 -- Exec'ing worker with command: "C:\Users\trist\cs_projects\Cancer_Project\Cancer_ML\env\python.exe" C:\Users\trist\cs_projects\Cancer_Project\Cancer_ML\env\lib\site-packages\ray\workers/default_worker.py --node-ip-address=127.0.0.1 --node-manager-port=56206 --object-store-name=tcp://127.0.0.1:59257 --raylet-name=tcp://127.0.0.1:62730 --redis-address=None --storage=None --temp-dir=C:\Users\trist\AppData\Local\Temp\ray --metrics-agent-port=59335 --logging-rotate-bytes=536870912 --logging-rotate-backup-count=5 --gcs-address=127.0.0.1:57228 --redis-password=5241590000000000 --startup-token=8 --runtime-env-hash=-1570358313
2022-07-12 06:50:43,426	INFO trial_runner.py:803 -- starting train_func_22de3_00002


In [ ]:
model.test_model(X_test, y_test)